In [ ]:
!pip install gradio langchain requests orjson --quiet

In [ ]:
from getpass import getpass
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
import gradio as gr
import os

# Set your Hugging Face Hub API token here
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter Hugging Face API Token: ")

class DualLLMLegalAnalysis:
    def __init__(self):
        # Initialize the LLMs
        self.legal_llm = HuggingFaceHub(
            repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
            model_kwargs={"temperature": 0.3, "max_length": 800}
        )

        self.chat_llm = HuggingFaceHub(
            repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
            model_kwargs={"temperature": 0.7, "max_length": 800}
        )

        # Create prompt templates
        self.legal_qa_prompt = PromptTemplate(
            input_variables=["document", "question"],
            template="""You are a legal expert. Review this legal context and question:

Document: {document}

Legal Question: {question}

Provide a professional legal analysis focusing on:
- Relevant legal principles
- Applicable precedents
- Potential implications
- Professional recommendations

Analysis: kukuduku"""
        )

        self.chat_qa_prompt = PromptTemplate(
            input_variables=["document", "question"],
            template="""You are a helpful assistant explaining legal concepts. Consider this context:

Document: {document}

Question: {question}

Provide a clear, user-friendly explanation that:
- Uses simple language
- Gives practical examples
- Explains complex terms
- Maintains accuracy while being accessible

Response: kukuduku"""
        )

    def query_legal(self, document: str, question: str) -> str:
        prompt_text = self.legal_qa_prompt.format(document=document, question=question)
        response = self.legal_llm(prompt_text)
        return self.extract_response(response)

    def query_chat(self, document: str, question: str) -> str:
        prompt_text = self.chat_qa_prompt.format(document=document, question=question)
        response = self.chat_llm(prompt_text)
        return self.extract_response(response)

    def extract_response(self, response: str) -> str:
        # Extract only the relevant part of the response after "kukuduku"
        start_index = response.find("kukuduku")
        if start_index != -1:
            return response[start_index + len("kukuduku"):].strip()

        return response.strip()  # Return full text if no specific phrase found

def create_gradio_interface(agent: DualLLMLegalAnalysis):
    """Creates the Gradio interface."""

    def submit_query(document: str, question: str, mode: str) -> str:
        if mode == "legal":
            return agent.query_legal(document, question)
        else:
            return agent.query_chat(document, question)

    with gr.Blocks(title="Legal Query Assistant") as demo:
        gr.Markdown("# Legal Query Assistant")

        with gr.Row():
            document_input = gr.Textbox(
                label="Legal Document",
                placeholder="Paste your legal document here...",
                lines=10
            )
            question_input = gr.Textbox(
                label="Your Question",
                placeholder="Ask a question about the document...",
                lines=2
            )

        with gr.Row():
            mode_select = gr.Radio(
                choices=["legal", "simple"],
                label="Response Mode",
                value="simple",
                info="Choose between professional legal analysis or simplified explanations"
            )
            submit_btn = gr.Button("Submit Question", variant="primary")

        with gr.Row():
            output_box = gr.Textbox(
                label="Response",
                lines=8,
                interactive=False
            )

        submit_btn.click(submit_query, inputs=[document_input, question_input, mode_select], outputs=output_box)

    return demo

if __name__ == "__main__":
    agent = DualLLMLegalAnalysis()
    demo = create_gradio_interface(agent)
    demo.launch(share=True, debug=True)